In [1]:
# Import necessary libraries and packages
import math
import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from qiskit import Aer, IBMQ, QuantumCircuit
from qiskit.primitives import Estimator
from qiskit.providers.aer import StatevectorSimulator
from qiskit.utils import QuantumInstance

from qiskit.tools.jupyter import *
from qiskit.visualization import *

# Import Qiskit libraries for VQE
from qiskit.algorithms.optimizers import SLSQP, SPSA

# Import Qiskit Nature libraries
from qiskit_nature.algorithms import GroundStateEigensolver, VQEUCCFactory
from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory
from qiskit_nature.circuit.library import UCC, UCCSD
from qiskit_nature.drivers import Molecule
from qiskit_nature.drivers.second_quantization import ElectronicStructureDriverType, ElectronicStructureMoleculeDriver
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import BravyiKitaevMapper, JordanWignerMapper, ParityMapper
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit_nature.transformers.second_quantization.electronic import ActiveSpaceTransformer

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.transformers import FreezeCoreTransformer


from qiskit.primitives import Estimator
from qiskit.circuit.library import EvolvedOperatorAnsatz
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock


In [2]:
molecule = MoleculeInfo(["H", "H"], [(0.0, 0.0, 0.0), (0.0, 0.0, 0.735)], charge=0, multiplicity=1)

driver = PySCFDriver.from_molecule(molecule, basis="sto3g")


problem = driver.run()

transformer = FreezeCoreTransformer()


problem = transformer.transform(problem)

hamiltonian_sq_op = problem.hamiltonian.second_q_op()
print(hamiltonian_sq_op)

Fermionic Operator
number spin orbitals=4, number terms=36
  -1.2563390730032498 * ( +_0 -_0 )
+ -0.47189600728114234 * ( +_1 -_1 )
+ -1.2563390730032498 * ( +_2 -_2 )
+ -0.47189600728114234 * ( +_3 -_3 )
+ 0.33785507740175813 * ( +_0 +_0 -_0 -_0 )
+ 0.09046559989211565 * ( +_0 +_0 -_1 -_1 )
+ 0.09046559989211556 * ( +_0 +_1 -_0 -_1 )
+ 0.33229086512764827 * ( +_0 +_1 -_1 -_0 )
+ 0.33785507740175813 * ( +_0 +_2 -_2 -_0 )
+ 0.09046559989211564 * ( +_0 +_2 -_3 -_1 )
+ 0.09046559989211556 * ( +_0 +_3 -_2 -_1 )
+ 0.33229086512764816 * ( +_0 +_3 -_3 -_0 )
+ 0.33229086512764816 * ( +_1 +_0 -_0 -_1 )
+ 0.09046559989211574 * ( +_1 +_0 -_1 -_0 )
+ 0.09046559989211564 * ( +_1 +_1 -_0 -_0 )
+ 0.34928686136600906 * ( +_1 +_1 -_1 -_1 )
+ 0.33229086512764827 * ( +_1 +_2 -_2 -_1 )
+ 0.09046559989211574 * ( +_1 +_2 -_3 -_0 )
+ 0.09046559989211565 * ( +_1 +_3 -_2 -_0 )
+ 0.34928686136600906 * ( +_1 +_3 -_3 -_1 )
+ 0.33785507740175813 * ( +_2 +_0 -_0 -_2 )
+ 0.09046559989211565 * ( +_2 +_0 -_1 -_3 )
+ 0

In [3]:
from qiskit_nature.second_q.mappers import JordanWignerMapper

mapper = JordanWignerMapper()
converter = QubitConverter(mapper)
qubit_op = converter.convert(hamiltonian_sq_op)
print(qubit_op)

-0.8105479805373271 * IIII
+ 0.1721839326191555 * IIIZ
- 0.2257534922240237 * IIZI
+ 0.1721839326191552 * IZII
- 0.22575349222402372 * ZIII
+ 0.12091263261776633 * IIZZ
+ 0.16892753870087907 * IZIZ
+ 0.045232799946057826 * YYYY
+ 0.045232799946057826 * XXYY
+ 0.045232799946057826 * YYXX
+ 0.045232799946057826 * XXXX
+ 0.16614543256382408 * ZIIZ
+ 0.1661454325638241 * IZZI
+ 0.1746434306830045 * ZIZI
+ 0.12091263261776633 * ZZII


In [4]:
from qiskit_nature.second_q.mappers import ParityMapper

converter = QubitConverter(ParityMapper(), two_qubit_reduction=True)
qubit_op = converter.convert(hamiltonian_sq_op, num_particles=problem.num_particles)
print(qubit_op)

-1.0523732457728594 * II
+ 0.39793742484317907 * IZ
- 0.39793742484317873 * ZI
- 0.01128010425623538 * ZZ
+ 0.18093119978423122 * XX


In [ ]:

estimator= Estimator()
ansatz = UCC(
    qubit_converter=converter,
    num_particles=problem.num_particles, 
    num_spin_orbitals=problem.num_spin_orbitals,
    excitations='sd'
)

optimizer = SLSQP(maxiter=100)


vqe = VQE(
    estimator, ansatz, optimizer
)

result_h2 = vqe.compute_minimum_eigenvalue(operator=qubit_op)
energy_h2= result_h2.eigenvalue.real
print(f"Energy estimated of hydrogen molecule using VQE: {energy_h2}")